# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('Churn.csv')

In [3]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В нашем распоряжении датасет, состоящий из 10000 строк и 14 столбцов.

В столбце Tenure имеются пропуски, причина пропусков неизвестна. Заменим пропуски на медианное значение.

In [5]:
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())

Столбцы RowNumber, CustomerId и Surname нужно удалить, так как эти данные не влияют на отток клиентов из банка.

In [6]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

Посмтрим, признаки каких типов хранятся в датафрейме

In [7]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure             float64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

Преобразуем категориальные признаки (Geography и Gender) в численные техникой прямого кодирования (OHE), избегая дамми-ловушки.

In [8]:
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


Спрятанной тестовой выборки нет, поэтому иммеющиеся данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разобъём в соотношении 3:1:1 или 60%:20%:20%.

In [9]:
features = df_ohe.drop('Exited', axis=1)
target = df_ohe['Exited']

In [10]:
features_train, features_validtest, target_train, target_validtest = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_validtest, target_validtest, test_size=0.5, random_state=12345)

In [11]:
print(features_train.shape) 
print(target_train.shape) 
print(features_valid.shape) 
print(target_valid.shape) 
print(features_test.shape) 
print(target_test.shape)

(6000, 11)
(6000,)
(2000, 11)
(2000,)
(2000, 11)
(2000,)


Стандартизируем численные признаки. 

In [12]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [13]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Исследование задачи

Исследуем баланс классов

In [14]:
target.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Мы видим сильный дисбаланс классов, но для начала обучим 3 модели без учета дисбаланса

**Decision Tree**

In [15]:
dt = DecisionTreeClassifier(random_state=12345)
parameters = {
    'max_depth': list(range(1, 50))
}
CV_dt = GridSearchCV(estimator=dt, param_grid=parameters, cv=5)
CV_dt.fit(features_train, target_train)
predicted_valid = CV_dt.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5320813771517998
AUC-ROC: 0.6872304453813536


**Random Forest**

In [16]:
rf = RandomForestClassifier(random_state=12345)
parameters = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_depth': list(range(1, 30))
}
CV_rf = GridSearchCV(estimator=rf, param_grid=parameters, cv=5)
CV_rf.fit(features_train, target_train)
predicted_valid = CV_rf.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5627980922098569
AUC-ROC: 0.7009765967614128


**Logistic Regression**

In [17]:
LR = LogisticRegression(random_state=12345, solver = 'lbfgs')
LR.fit(features_train, target_train)
predicted_valid = LR.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.33108108108108103
AUC-ROC: 0.5932046528227246


**Самый высокий показатель F1 оказался у модели Random forest (F1=0.56)**

## Борьба с дисбалансом

Для борьбы с дисбалансом мы можем воспользоваться техникой upsampling/downsampling и взвешивания классов.

Проверим технику увеличения выборки.

In [18]:
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

Теперь посмотрим модели после увеличения выборки

**Decision Tree**

In [19]:
dt_upsampled = DecisionTreeClassifier(random_state=12345)
parameters = {
    'max_depth': list(range(1, 50))
}
CV_dt_upsampled = GridSearchCV(estimator=dt_upsampled, param_grid=parameters, cv=5)
CV_dt_upsampled.fit(features_upsampled, target_upsampled)
predicted_valid = CV_dt_upsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.49033816425120774
AUC-ROC: 0.6773994519686183


**Random Forest**

In [20]:
rf_upsampled = RandomForestClassifier(random_state=12345)
parameters = {
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200],
    'max_depth': list(range(1, 30))
}
CV_rf_upsampled = GridSearchCV(estimator=rf_upsampled, param_grid=parameters, cv=5, n_jobs=-1)
CV_rf_upsampled.fit(features_upsampled, target_upsampled)
predicted_valid = CV_rf_upsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5952380952380952
AUC-ROC: 0.7334244702665755


**Logistic Regression**

In [21]:
LR_upsampled = LogisticRegression(random_state=12345, solver = 'lbfgs')
LR_upsampled.fit(features_upsampled, target_upsampled)
predicted_valid = LR_upsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.4888507718696398
AUC-ROC: 0.6945753361682564


**Максимальный F1 получился у модели Random Forest (F1=0.59)**

Попробуем downsampling

In [22]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

**Decision Tree**

In [23]:
dt_downsampled = DecisionTreeClassifier(random_state=12345)
parameters = {'max_depth': list(range(1, 50))}
CV_dt_downsampled = GridSearchCV(estimator=dt_downsampled, param_grid=parameters, cv=5)
CV_dt_downsampled.fit(features_downsampled, target_downsampled)
predicted_valid = CV_dt_downsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5749525616698292
AUC-ROC: 0.7571936679994435


**Random Forest**

In [24]:
rf_downsampled = RandomForestClassifier(random_state=12345)
parameters = {
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200],
    'max_depth': list(range(1, 30))
}
CV_rf_downsampled = GridSearchCV(estimator=rf_downsampled, param_grid=parameters, cv=5, n_jobs=-1)
CV_rf_downsampled.fit(features_downsampled, target_downsampled)
predicted_valid = CV_rf_downsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5990521327014218
AUC-ROC: 0.7765365747433749


**Logistic Regression**

In [25]:
LR_downsampled = LogisticRegression(random_state=12345, solver = 'lbfgs', class_weight='balanced')
LR_downsampled.fit(features_downsampled, target_downsampled)
predicted_valid = LR_downsampled.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.4866723989681858
AUC-ROC: 0.6924990472964391


**После уменьшения выборки максимальный F1 снова получился у модели Random Forest (F1=0.59)**

Теперь попробуем добавить аргумент class_weight='balanced'в Random Forest downsampled, так как у этой модели в итоге получился самый высокий F1.

In [26]:
rf_downsampled_balanced = RandomForestClassifier(random_state=12345)
parameters = {
    'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 100, 200],
    'max_depth': list(range(1, 30))
}
CV_rf_downsampled_balanced = GridSearchCV(estimator=rf_downsampled_balanced, param_grid=parameters, cv=5, n_jobs=-1)
CV_rf_downsampled_balanced.fit(features_downsampled, target_downsampled)
predicted_valid = CV_rf_downsampled_balanced.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, predicted_valid))

F1: 0.5990521327014218
AUC-ROC: 0.7765365747433749


Можно еще попробовать изменить порог.

In [27]:
probabilities_valid = CV_rf_downsampled_balanced.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:,1]
thrs = [0] + sorted(list(probabilities_one_valid))
res = []
for thr in thrs:
    predicted_valid = (probabilities_one_valid > thr)*1
    res.append((thr, f1_score(target_valid, predicted_valid)))
    
t = pd.DataFrame(res,columns=['thr','F1'])
t[t['F1'] == t['F1'].max()]

,thr,F1
1534,0.58824,0.626697


In [28]:
threshold = 0.58824

predicted_proba = CV_rf_downsampled_balanced.predict_proba(features_valid)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

print('F1:', f1_score(target_valid, predicted))
print('AUC-ROC:', roc_auc_score(target_valid, predicted))

F1: 0.6266968325791856
AUC-ROC: 0.771605199644324


## Тестирование модели

In [29]:
predictions_test = CV_rf_downsampled_balanced.predict(features_test)
print('F1:', f1_score(target_test, predictions_test))
print('AUC-ROC:', roc_auc_score(target_test, predictions_test))

F1: 0.57380073800738
AUC-ROC: 0.7566420965684313


In [30]:
threshold = 0.58824

predicted_proba = CV_rf_downsampled_balanced.predict_proba(features_test)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

print('F1:', f1_score(target_test, predicted))
print('AUC-ROC:', roc_auc_score(target_test, predicted))

F1: 0.5995525727069352
AUC-ROC: 0.7524221559624087


Попробуем объединить валидационную и тренировочную выборку в одну обучающую, это может дать нам еще немного качества.

In [31]:
features_train_valid = pd.concat([features_train, features_valid])
target_train_valid = pd.concat([target_train, target_valid])

CV_rf_downsampled_balanced.fit(features_train_valid, target_train_valid) 

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 100,
                                          200]})

In [32]:
predictions_test2 = CV_rf_downsampled_balanced.predict(features_test)
print('F1:', f1_score(target_test, predictions_test2))
print('AUC-ROC:', roc_auc_score(target_test, predictions_test2))

F1: 0.5248447204968943
AUC-ROC: 0.6832765927464992


In [33]:
threshold2 = 0.58824

predicted_proba2 = CV_rf_downsampled_balanced.predict_proba(features_test)
predicted = (predicted_proba [:,1] >= threshold2).astype('int')

print('F1:', f1_score(target_test, predicted))
print(f'AUC-ROC:', roc_auc_score(target_test, predicted))

F1: 0.5995525727069352
AUC-ROC: 0.7524221559624087


Лучшей моделью для решения поставленной задачи оказалась модель случайного леса. Для борьбы с бисбалансом классов была использована техника downsampling. Основная цель достигнута: F1 > 0.59